# DLA for Z2 lattice gauge theory Hamiltonian variational ansatz (sparse implementation)

In [1]:
import numpy as np
import h5py
from fastdla.dla import generate_dla, is_independent
from fastdla.ansatze.z2lgt_hva import (z2lgt_hva_generators, z2lgt_gauss_projector,
                                       z2lgt_u1_projector, z2lgt_dense_projector)

In [2]:
data_file = 'z2lgt_hva_dla.h5'

## 2 Fermions

In [3]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
charge = 0

In [ ]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

In [ ]:
# Compute the DLA of the full space
dla_full = generate_dla(generators_full, min_tasks=200, max_workers=54, verbosity=2)
print(f'DLA dimension is {len(dla_full)}')

In [5]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, charge)
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [ ]:
# Check the dimensionality of the symmetry subspace
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')

In [ ]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, min_tasks=200, max_workers=54, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

In [10]:
with h5py.File(data_file, 'a') as out:
    if 'nf=2' not in out:
        nf_group = out.create_group('nf=2')
        for gname, oplist in [
            ('generators_full', generators_full),
            ('dla_full', dla_full),
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('indices', data=np.concatenate([op.indices for op in oplist]))
            group.create_dataset('coeffs', data=np.concatenate([op.coeffs for op in oplist]))
            group.create_dataset('indptr', data=np.cumsum([0] + [op.num_terms for op in oplist]))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)

## 3 Fermions

In [11]:
num_fermions = 3
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1]
charge = 0

In [ ]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

In [13]:
# Compute the DLA of the full space
# dla_full = generate_dla(generators_full, min_tasks=100, max_workers=54, verbosity=3)
# print(f'DLA dimension is {len(dla_full)}')

In [14]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, charge)
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [ ]:
# Check the dimensionality of the symmetry subspace
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')

In [ ]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=2, min_tasks=200, max_workers=54)
print(f'Subspace DLA dimension is {len(dla)}')

In [ ]:
with h5py.File(data_file, 'a') as out:
    if 'nf=3' not in out:
        nf_group = out.create_group('nf=3')
        for gname, oplist in [
            ('generators_full', generators_full),
            # ('dla_full', dla_full),
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('indices', data=np.concatenate([op.indices for op in oplist]))
            group.create_dataset('coeffs', data=np.concatenate([op.coeffs for op in oplist]))
            group.create_dataset('indptr', data=np.cumsum([0] + [op.num_terms for op in oplist]))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)